In [6]:
import pandas as pd
import numpy as np

# 원본 데이터 로드
df = pd.read_csv('../data/raw/online_retail.csv')
print(df.shape)

# 1. 날짜 타입 변환 
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# 2. 음수 단가 제거
# EDA를 통한 문제 인식: 가격(UnitPrice)이 음수일 수 없음

# 2-1. 단순 오타(마이너스 기호) 인지 확인
negative_price = df[df['UnitPrice'] < 0]
print(negative_price[['InvoiceNo', 'StockCode', 'Description','Quantity', 'UnitPrice', 'CustomerID']])

# 확인 결과: InvoiceNo가 A로 시작(EDA 시 A로 시작하는 송장 3건 발견했었음)
# StockCode: B,  Description: 'Adjust bad debt' -> 회계 조정 항목으로 판단 
# 음수는 오타가 아니었고, 회계 조정 항목은 실제 상품 판매와는 관련 없는 장부 정리용 데이터이므로 제거 필요

# 2-2. 음수가 아닌 나머지 1건의 A도 회계 조정 항목인지 확인
a_invoices = df[df['InvoiceNo'].astype(str).str.startswith('A')]
print(a_invoices[['InvoiceNo', 'StockCode', 'Description','Quantity', 'UnitPrice', 'CustomerID']])
# A 3건 모두 회계 조정 항목 -> 제거 판단

# 2-3. A로 시작하는 회계 조정 항목 제거
is_adjustment = df['InvoiceNo'].astype(str).str.startswith('A')
df = df[~is_adjustment]

# 제거 검증
print(df['InvoiceNo'].astype(str).str.startswith('A').sum())  # 0 제거 완료
print((df['UnitPrice'] < 0).sum()) # 0 제거 완료

# 3. 그 외의 NULL 그대로 유지
# - CustomerID NULL → 유지 (비회원 구매일 수 있음)
# - Description NULL → 유지 (상품명 누락일 수 있음)
# SQL 분석 시 목적에 따라 뷰에서 처리

# 4. StockCode 대소문자 - 일단 보존
# df['StockCode'] = df['StockCode'].str.upper()  # 주석 처리
# SQL에서 필요시 UPPER() 사용

# 5. 저장 
output_path = '../data/cleaned/cleaned_data.csv'
df.to_csv(output_path, index=False)

# 6. 요약
print(f"최종 레코드: {len(df):,}건")

(541909, 8)
       InvoiceNo StockCode      Description  Quantity  UnitPrice  CustomerID
299983   A563186         B  Adjust bad debt         1  -11062.06         NaN
299984   A563187         B  Adjust bad debt         1  -11062.06         NaN
       InvoiceNo StockCode      Description  Quantity  UnitPrice  CustomerID
299982   A563185         B  Adjust bad debt         1   11062.06         NaN
299983   A563186         B  Adjust bad debt         1  -11062.06         NaN
299984   A563187         B  Adjust bad debt         1  -11062.06         NaN
0
0
최종 레코드: 541,906건
